In [1]:
!pip install bert-score
!gdown --id '1lovF1aJoy6IV2LpfeKO-NrObVGjED2Mv'
!pip install googletrans==3.1.0a0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.9 MB/s eta 0:00:00
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1lovF1aJoy6IV2LpfeKO-NrObVGjED2Mv
To: /content/hcm.xlsx
100% 46.7M/46.7M [00:00<00:00, 73.0MB/s]
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
from googletrans import Translator
import bert_score
import torch
translator = Translator()

In [3]:
def append_record_to_excel(file_path, question,answer,fa_question,fa_answer,
                           bt_question,bt_answer,question_P, question_R,
                           question_F1,answer_P, answer_R, answer_F1):
    new_record = {
        'question': question,
        'answer': answer,
        'fa_question':fa_question,
        'fa_answer':fa_answer,
        'bt_question':bt_question,
        'bt_answer': bt_answer,
        'question_P': question_P,
        'question_R': question_R,
        'question_F1': question_F1,
        'answer_P': answer_P,
        'answer_R': answer_R,
        'answer_F1':answer_F1
    }
    new_record_df = pd.DataFrame([new_record])
    try:
        existing_df = pd.read_excel(file_path)
        updated_df = pd.concat([existing_df, new_record_df], ignore_index=True)
    except FileNotFoundError:
        updated_df = new_record_df

    updated_df.to_excel(file_path, index=False)

In [ ]:
df = pd.read_excel('hcm.xlsx')
bgn = 2013
df = df[bgn:]
device = 'cuda' if torch.cuda.is_available() else 'cpu'
for index,row in df.iterrows():
    question = row['question']
    answer = row['answer']
    fa_question = translator.translate(question, dest='fa').text
    fa_answer = translator.translate(answer, dest='fa').text
    bt_question = translator.translate(fa_question, dest='en').text
    bt_answer = translator.translate(fa_answer, dest='en').text
    question_P, question_R, question_F1 = bert_score.score([bt_question], [question], lang='en', device=device, verbose=True)
    answer_P, answer_R, answer_F1 = bert_score.score([bt_answer], [answer], lang='en', device=device, verbose=True)
    append_record_to_excel('translated_hcm.xlsx', question,answer,fa_question,fa_answer,
                           bt_question,bt_answer,question_P.item(), question_R.item(),
                           question_F1.item(),answer_P.item(), answer_R.item(), answer_F1.item())